In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
##import tensorflow as tf



from  tensorflow.keras.preprocessing.text import Tokenizer
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('fin_data.csv',delimiter=',',encoding='latin-1')
df.head()

,neutral,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing ."
0,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...


In [ ]:
df = df.rename(columns={'neutral':'sentiment','According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .':'Message'})

In [ ]:
df.head()

,sentiment,Message
0,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...


In [ ]:
sentiment  = {'positive': 0,'neutral': 1,'negative':2}

df.sentiment = [sentiment[item] for item in df.sentiment]
print(df)

      sentiment                                            Message
0             1  Technopolis plans to develop in stages an area...
1             2  The international electronic industry company ...
2             0  With the new production plant the company woul...
3             0  According to the company 's updated strategy f...
4             0  FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
...         ...                                                ...
4840          2  LONDON MarketWatch -- Share prices ended lower...
4841          1  Rinkuskiai 's beer sales fell by 6.5 per cent ...
4842          2  Operating profit fell to EUR 35.4 mn from EUR ...
4843          2  Net sales of the Paper segment decreased to EU...
4844          2  Sales in Finland decreased by 10.5 % in Januar...

[4845 rows x 2 columns]


In [ ]:
from bs4 import BeautifulSoup
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text)
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text
df['Message'] = df['Message'].apply(cleanText)

<ipython-input-6-aaa97489b476>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "lxml").text


In [ ]:
print(df)

      sentiment                                            Message
0             1  technopolis plans to develop in stages an area...
1             2  the international electronic industry company ...
2             0  with the new production plant the company woul...
3             0  according to the company 's updated strategy f...
4             0  financing of aspocomp 's growth aspocomp is ag...
...         ...                                                ...
4840          2  london marketwatch -- share prices ended lower...
4841          1  rinkuskiai 's beer sales fell by 6.5 per cent ...
4842          2  operating profit fell to eur 35.4 mn from eur ...
4843          2  net sales of the paper segment decreased to eu...
4844          2  sales in finland decreased by 10.5 % in januar...

[4845 rows x 2 columns]


In [ ]:
train, test = train_test_split(df, test_size=0.000001 , random_state=42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            #if len(word) < 0:
            if len(word) <= 0:
                continue
            tokens.append(word.lower())
    return tokens
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['Message']), tags=[r.sentiment]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['Message']), tags=[r.sentiment]), axis=1)

# The maximum number of words to be used. (most frequent)
max_fatures = 500000

# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 50

#tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer = Tokenizer(num_words=max_fatures, split=' ', filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Message'].values)
X = tokenizer.texts_to_sequences(df['Message'].values)
X = pad_sequences(X)
print('Found %s unique tokens.' % len(X))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Found 4845 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(df['Message'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (4845, 50)


In [ ]:
d2v_model = Doc2Vec(dm=1, dm_mean=1, vector_size=20, window=8, min_count=1, workers=1, alpha=0.065, min_alpha=0.065) # Changed 'size' to 'vector_size'
d2v_model.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 4844/4844 [00:00<00:00, 1788329.25it/s]


In [ ]:
%%time
for epoch in range(30):
    d2v_model.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    d2v_model.alpha -= 0.002
    d2v_model.min_alpha = d2v_model.alpha

100%|██████████| 4844/4844 [00:00<00:00, 1620967.65it/s]


CPU times: user 9.76 s, sys: 123 ms, total: 9.88 s
Wall time: 10.2 s


In [ ]:
# save the vectors in a new matrix
embedding_matrix = np.zeros((len(d2v_model.wv.key_to_index) + 1, 20))

# Iterate through document vectors using d2v_model.dv
for i in range(len(d2v_model.dv)):
    # Ensure index is within embedding matrix bounds
    if i < embedding_matrix.shape[0]:
        embedding_matrix[i] = d2v_model.dv[i]

In [ ]:
d2v_model.wv.most_similar(positive=['profit'], topn=10)

[('loss', 0.9208242297172546),
 ('bill', 0.8321678638458252),
 ('profits', 0.8218406438827515),
 ('sales', 0.7571663856506348),
 ('income', 0.7457572817802429),
 ('chairman', 0.7304867506027222),
 ('aspokem', 0.7225751280784607),
 ('atul', 0.7169831991195679),
 ('walby', 0.7161943912506104),
 ('searching', 0.7005779147148132)]

In [ ]:
d2v_model.wv.most_similar(positive=['investment'], topn=10)

[('op-pohjola', 0.784252405166626),
 ('veracel', 0.7734625935554504),
 ('pension', 0.768334686756134),
 ('property', 0.7577870488166809),
 ('organization', 0.7368638515472412),
 ('rohwedder', 0.7363954782485962),
 ('8,000', 0.7325825691223145),
 ('+_land', 0.7185679078102112),
 ('limited', 0.7049908638000488),
 ('42.9', 0.7042791843414307)]

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding


# init layer
model = Sequential()

# emmbed word vectors
# Use key_to_index instead of vocab
model.add(Embedding(len(d2v_model.wv.key_to_index)+1,20,input_length=X.shape[1],weights=[embedding_matrix],trainable=True))

# learn the correlations
def split_input(sequence):
     return sequence[:-1], tf.reshape(sequence[1:], (-1,1))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(3,activation="softmax"))

# output model skeleton
model.summary()
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['acc'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │         233,660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 233,660 (912.73 KB)

 Trainable params: 233,660 (912.73 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
Y = pd.get_dummies(df['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.15, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(4118, 50) (4118, 3)
(727, 50) (727, 3)


In [ ]:
batch_size = 32
history=model.fit(X_train, Y_train, epochs =50, batch_size=batch_size, verbose = 2)

Epoch 1/50
129/129 - 9s - 67ms/step - acc: 0.5867 - loss: 0.5522
Epoch 2/50
129/129 - 1s - 8ms/step - acc: 0.6501 - loss: 0.4789
Epoch 3/50
129/129 - 1s - 10ms/step - acc: 0.7530 - loss: 0.3533
Epoch 4/50
129/129 - 1s - 10ms/step - acc: 0.8176 - loss: 0.2643
Epoch 5/50
129/129 - 1s - 10ms/step - acc: 0.8407 - loss: 0.2240
Epoch 6/50
129/129 - 1s - 8ms/step - acc: 0.8715 - loss: 0.1856
Epoch 7/50
129/129 - 1s - 7ms/step - acc: 0.9208 - loss: 0.1370
Epoch 8/50
129/129 - 1s - 5ms/step - acc: 0.9597 - loss: 0.0862
Epoch 9/50
129/129 - 1s - 7ms/step - acc: 0.9633 - loss: 0.0740
Epoch 10/50
129/129 - 1s - 10ms/step - acc: 0.9772 - loss: 0.0512
Epoch 11/50
129/129 - 2s - 13ms/step - acc: 0.9864 - loss: 0.0320
Epoch 12/50
129/129 - 2s - 16ms/step - acc: 0.9852 - loss: 0.0346
Epoch 13/50
129/129 - 1s - 5ms/step - acc: 0.9900 - loss: 0.0260
Epoch 14/50
129/129 - 1s - 9ms/step - acc: 0.9920 - loss: 0.0207
Epoch 15/50
129/129 - 1s - 10ms/step - acc: 0.9920 - loss: 0.0175
Epoch 16/50
129/129 - 1s -

In [ ]:
_, train_acc = model.evaluate(X_train, Y_train, verbose=2)
_, test_acc = model.evaluate(X_test, Y_test, verbose=2)
print('Train: %.3f, Test: %.4f' % (train_acc, test_acc))

129/129 - 1s - 5ms/step - acc: 0.9993 - loss: 0.0019
23/23 - 0s - 3ms/step - acc: 0.7235 - loss: 1.1058
Train: 0.999, Test: 0.7235


In [ ]:
import numpy as np # Import numpy for argmax

# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
print(yhat_probs)
# predict crisp classes for test set
# Use predict and argmax to get predicted classes
yhat_classes = np.argmax(model.predict(X_test, verbose=0), axis=-1)
print(yhat_classes)
# reduce to 1d array
#yhat_probs = yhat_probs[:, 0]
#yhat_classes = yhat_classes[:, 1

[[5.92666822e-07 9.99999166e-01 2.12400607e-07]
 [9.99999642e-01 1.04761725e-07 2.39127814e-07]
 [3.19498596e-08 1.00000000e+00 2.93649549e-08]
 ...
 [9.99754250e-01 1.63544013e-04 8.22444345e-05]
 [1.22674990e-08 1.00000000e+00 1.10872449e-08]
 [9.99999523e-01 2.92485197e-07 1.84612418e-07]]
[1 0 1 0 1 1 1 1 1 1 1 1 0 2 1 1 1 1 0 1 1 1 0 0 0 0 1 1 0 2 1 1 1 2 1 0 2
 0 1 1 0 1 2 1 0 1 1 1 0 1 2 1 0 2 1 0 1 1 0 1 0 0 0 0 1 1 1 1 0 2 1 0 0 1
 1 1 1 1 1 0 1 2 0 0 0 0 1 0 1 0 0 1 1 2 1 1 0 1 0 1 2 1 1 2 0 1 1 1 2 1 1
 1 2 1 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 2 1
 2 1 1 0 1 1 1 0 0 1 2 1 1 1 1 0 1 0 1 2 1 0 1 0 0 1 0 1 0 0 1 0 1 0 1 1 1
 2 1 0 1 1 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 0 1 1 2 0 0 0 1
 1 0 1 1 1 0 1 1 1 1 1 1 2 0 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1
 1 1 0 1 0 2 2 0 1 1 1 1 1 2 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 0 1 0 1 0 1 2 1
 1 2 0 1 0 1 1 1 1 2 1 0 1 0 1 2 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1
 1 0 1 1 0 0 1 0 1 0 0 1 1 1 0 

In [ ]:
message = ['Congratulations! you have won a $1,000 Walmart gift card']
seq = tokenizer.texts_to_sequences(message)

padded = pad_sequences(seq, maxlen=X.shape[1], dtype='int32', value=0)

pred = model.predict(padded)

labels = ['0','1','2']
print(pred, labels[np.argmax(pred)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
[[0.99118656 0.00650035 0.00231306]] 0


In [34]:
message = ['']
seq = tokenizer.texts_to_sequences(message)

padded = pad_sequences(seq, maxlen=X.shape[1], dtype='int32', value=0)

pred = model.predict(padded)

labels = ['0','1','2']
print(pred, labels[np.argmax(pred)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[0.02287179 0.9715707  0.00555746]] 1


In [35]:
pip install requests beautifulsoup4 lxml

In [50]:
import requests
import numpy as np
from keras.preprocessing.sequence import pad_sequences

# Define the API key and endpoint
api_key = 'YOUR API KEY'
url = 'https://cryptopanic.com/api/v1/posts/'

# Define parameters for the API request
params = {
    'auth_token': api_key,
    'filter': 'bullish',  # You can change this to 'bullish', 'bearish', 'important', etc.
    'public': 'true'  # Fetch only public news
}

# Define sentiment labels (adjusted to match your encoded values)
sentiment = {0: 'positive', 1: 'neutral', 2: 'negative'}

# Initialize counters for sentiment analysis
sentiment_counts = {'positive': 0, 'neutral': 0, 'negative': 0}
total_posts = 0

# Make the API request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Loop through each post and perform sentiment analysis
    for post in data['results']:
        message = [post['title']]  # Using the post's title for sentiment analysis
        seq = tokenizer.texts_to_sequences(message)

        # Ensure the input has the same shape as the model's expected input
        padded = pad_sequences(seq, maxlen=X.shape[1], dtype='int32', value=0)

        # Predict sentiment
        pred = model.predict(padded)

        # Get the predicted sentiment based on your encoding
        predicted_label = np.argmax(pred)
        sentiment_label = sentiment[predicted_label]

        # Increment the corresponding sentiment count
        sentiment_counts[sentiment_label] += 1
        total_posts += 1

    # Calculate average sentiment by percentage
    positive_pct = (sentiment_counts['positive'] / total_posts) * 100 if total_posts > 0 else 0
    neutral_pct = (sentiment_counts['neutral'] / total_posts) * 100 if total_posts > 0 else 0
    negative_pct = (sentiment_counts['negative'] / total_posts) * 100 if total_posts > 0 else 0

    # Print the sentiment counts and percentages
    print(f"Total Posts Analyzed: {total_posts}")
    print(f"Positive: {sentiment_counts['positive']} ({positive_pct:.2f}%)")
    print(f"Neutral: {sentiment_counts['neutral']} ({neutral_pct:.2f}%)")
    print(f"Negative: {sentiment_counts['negative']} ({negative_pct:.2f}%)")

    # Calculate average sentiment score (-1 for negative, 0 for neutral, +1 for positive)
    average_sentiment_score = (sentiment_counts['positive'] - sentiment_counts['negative']) / total_posts if total_posts > 0 else 0

    # Print average sentiment score (between -1 and 1)
    print(f"Average Sentiment Score: {average_sentiment_score:.2f}")

else:
    print(f"Failed to fetch data. Status code: {response.status_code}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Total Posts Analyzed: 20
Positive: 6 (30.00%)
Neutral: 13 (65.00%)
Negative: 1 (5.00%)
Average Sentiment Score: 0.25
